In [ ]:
from QuantitativeCleaner import *

In [ ]:
# Read Path
optimized_path = "F:/Data/Processing_data/OptimizedTrackData.parquet"

# Write paths
group_dir = "F:/Data/Processing_data/injury_output"
main_dir = "F:/Data/Processing_data/"

In [ ]:
# optimize_injury_data()

In [ ]:
# process_file(optimized_path, group_dir)

In [ ]:
#### Tracking Output
def tracking_injuries(group_dir, main_dir):
    from DataHandler import data_loader
    import os
    import polars as pl # type: ignore
    pl.enable_string_cache()

    # Read in the PlayKeys from the injury file to isolate PlayKeys associated with injury paths
    injuryPlayKeys = data_loader('injuries')
    PlayKeys = injuryPlayKeys.select("PlayKey").unique().with_columns([
            pl.col("PlayKey").cast(pl.Utf8)
            ])

    # Initialize an empty list to store dataframes
    filtered_dfs = []

    # Iterate through files in the directory
    for file in os.listdir(group_dir):
        if file.startswith("group_"):
            file_path = os.path.join(group_dir, file)
            
            # Read the Parquet file
            df = pl.read_parquet(file_path)
            
            # Ensure PlayKey is of type Utf8
            df = df.with_columns(pl.col("PlayKey").cast(pl.Utf8))
            
            # Inner join with unique_gsisid to filter rows
            filtered_df = df.join(PlayKeys, on="PlayKey", how="inner")
            
            # Append to the list of filtered dataframes
            filtered_dfs.append(filtered_df)

    # Concatenate all filtered dataframes
    final_df = pl.concat(filtered_dfs)

    # Save the concatenated dataframe
    final_df.write_parquet(os.path.join(main_dir, "TrackingInjuries.parquet"))

    print("Processing complete. Filtered summary dataframe saved as 'TrackingInjuries.parquet'")

In [ ]:
tracking_injuries(group_dir, main_dir)

In [ ]:
from QualitativeCleaner import *
from DataHandler import parquet_writer, data_shrinker

In [ ]:
df = table_joiner('injury')
df = injury_interpolator(df, 'injury')
df = stadium_cleaner(df)
df = weather_cleaner(df)
df, schema = data_shrinker(df)


df.head()

In [ ]:
##### Primary Injury Cleaning Function #####
def clean_injury_qual():
    """
    Applies data cleaning to surface injury data and writes to 'qualitative_injuries' as a csv file 
    """
    from DataHandler import parquet_writer, data_shrinker
    import os 

    analysis = "injury"
    injury_qual_path = "F:/Data/Processing_data/QualitativeInjuries.parquet"


    df = table_joiner(analysis) 
    df = injury_interpolator(df, analysis)
    df = stadium_cleaner(df)
    df = weather_cleaner(df)
    df, schema = data_shrinker(df)
    df.write_parquet(injury_qual_path)

    print('Injuries have been cleaned and dressed.')
    # return df


In [ ]:
clean_injury_qual()

In [ ]:
##### Primary Concussion Cleaning Function #####
def clean_concussions(): 
    """
    Applies data cleaning to surface injury data and writes to 'qualitative_injuries' as a csv file 
    """
    from DataHandler import parquet_writer, data_shrinker

    analysis = "concussion"
    concussion_qual_path = "F:/Data/Processing_data/QualitativeConcussions.parquet"
    
    df = table_joiner(analysis)
    df = injury_interpolator(df, analysis)
    df = stadium_cleaner(df)
    df = weather_cleaner(df)
    df = turf_cleaner(df)
    df = cancellation_cleaner(df)
    df = score_splitter(df)
    df, schema = data_shrinker(df)
    df.write_parquet(concussion_qual_path)
    del df

    print('Concussions have been assessed and cleared for play.')
    # return df

In [ ]:
clean_concussions()

In [ ]:
from QuantitativeCleaner import *

In [ ]:
injury_summary_maker(group_dir = "F:/Data/Processing_data/injury_output")

In [ ]:
import polars as pl

len(pl.read_parquet("F:/Data/Processing_data/Full_Summary_Injuries.parquet"))

# Concussion Data

In [ ]:
df = pl.read_csv("F:/Data/NFL-Punt-Analytics-Competition/NGS-2017-post.csv")

In [ ]:
df.head()

In [ ]:
pl.read_parquet("F:/Data/Processing_data/TrackingInjuries.parquet").head()

In [ ]:
from QuantitativeCleaner import *
import polars as pl
import os


source_dir = "F:/Data/NFL-Punt-Analytics-Competition/"
concussion_dir = "F:/Data/Processing_data/concussion_output"

os.makedirs(concussion_dir, exist_ok=True)

In [ ]:
df_test, schema = data_shrinker(df)
df_test.head()

In [ ]:
def column_corrector(df):
    import polars as pl # type: ignore
    """
    Add a Play_Time column that acts like the 'time' column did in the injury dataset. 
    Each PlayKey will start at 0.0 and increase by 0.1 for each subsequent record.
    """
    df = df.with_columns([
        pl.concat_str([
            pl.col('GSISID').cast(pl.Int32).cast(pl.Utf8)
            , pl.lit('-')
            , pl.col('GameKey').cast(pl.Utf8)
            , pl.lit('-')
            , pl.col('PlayID').cast(pl.Utf8)
        ]).alias('PlayKey')
    ])
     
    
    df = df.select([
        'PlayKey'
        , 'Time'
        , 'x'
        , 'y'
        , 'o'
        , 'dir'
        , 'GSISID'
        ]).rename({"Time":"DateTime"})

    df = df.sort(['PlayKey', 'DateTime'])

    df = df.with_columns(
        (pl.arange(0, pl.len()) * 0.1).over("PlayKey").cast(pl.Int32).alias("time")
        ).with_columns([pl.col('GSISID').cast(pl.Int32)])  
    
    df = df.drop(['DateTime'])
    
    return df

In [ ]:
def body_builder_conc(df):
    """
    This uses averages collected for height, weight, and chest radius for each position. This information
    is used to determine the momentum and impulse rather than just looking at velocities in the analysis. Chest
    radius is needed for angular moment of inertia as a rotating cylinder.
    The data here are cast as f32 to reduce the size of these columns as well as in all future calculations, where the f64 
    gets exponentially larger with application. 
    """
    import polars as pl # type: ignore

    # Enable global string cache
    pl.enable_string_cache()


    try:
        body_data = pl.DataFrame({
            "Position": ["QB", "RB", "FB", "WR", "TE", "T", "G", "C", "DE", "DT", "NT", "LB", "OLB", "MLB", "CB", "S", "K", "P", "SS", "ILB", "FS", "LS", "DB"]
            , "Height_m": [1.91, 1.79, 1.85, 1.88, 1.96, 1.97, 1.90, 1.87, 1.97, 1.92, 1.88, 1.90, 1.90, 1.87, 1.82, 1.84, 1.83, 1.88, 1.84, 1.90, 1.84, 1.88, 1.82]
            , "Weight_kg": [102.1, 95.3, 111.1, 90.7, 114.6, 140.6, 141.8, 136.1, 120.2, 141.8, 152.0, 110.0, 108.9, 113.4, 87.4, 95.9, 92.08, 97.52, 95.9, 110.0, 95.9, 108.86, 87.4]
            , "Chest_rad_m": [0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191, 0.191]
        }).with_columns([
            pl.col("Height_m").cast(pl.Float32)
            , pl.col("Weight_kg").cast(pl.Float32)
            , pl.col("Chest_rad_m").cast(pl.Float32)
            , pl.col("Position").cast(pl.Categorical)
        ])

        
        Player_path = "F:/Data/NFL-Punt-Analytics-Competition/player_punt_data.csv"
        position = pl.read_csv(Player_path).select(["GSISID", "Position"])
        position = position.with_columns([
            pl.col("GSISID").cast(pl.Int32)
            , pl.col("Position").cast(pl.Categorical)
        ])

        position = position.join(
            body_data
            , on='Position'
            , how='left'
        )

        df = df.with_columns([
            pl.col("GSISID").cast(pl.Int32)
        ])

        df = df.join(
            position
            , on='GSISID'
            , how='left'
        )    

        return df.filter(pl.col('Position').is_not_null())    
        
    except Exception as e: 
        print(f"An error occurred during body_builder: {e}")
        return None

In [ ]:
review = pl.read_csv("F:/Data/NFL-Punt-Analytics-Competition/video_review.csv")

In [ ]:
def clean_review():
    review = pl.read_csv("F:/Data/NFL-Punt-Analytics-Competition/video_review.csv")
    review, schema = data_shrinker(review)
    review = review.with_columns([
            pl.concat_str([
                pl.col('GSISID').cast(pl.Int32).cast(pl.Utf8)
                , pl.lit('-')
                , pl.col('GameKey').cast(pl.Utf8)
                , pl.lit('-')
                , pl.col('PlayID').cast(pl.Utf8)
            ]).alias('PlayKey')
        ]).drop(['Season_Year', 'GameKey', 'PlayID', 'GSISID', 'Turnover_Related', 'Friendly_Fire'])
    
    return review

In [ ]:
review = clean_review()

In [ ]:
review.head()

In [ ]:
df_testes = df_test.join(
    review
    , on="PlayKey"
    , how="left"
)

In [ ]:
def add_review_data(df):
    review = clean_review()

    df = df.join(
        review
        , on="PlayKey"
        , how="left"    
        )
    
    return df

In [ ]:
df_test, schema = data_shrinker(df)
df_test = column_corrector(df_test)
df_test = angle_corrector(df_test)
df_test = body_builder_conc(df_test)
df_test = velocity_calculator(df_test)
df_test = impulse_calculator(df_test)
df_test = add_review_data(df_test)
df_test.head()

In [ ]:
group_df, schema = data_shrinker(df)
group_df = (group_df
                .pipe(column_corrector)
                .pipe(angle_corrector)
                .pipe(body_builder_conc)
                .pipe(velocity_calculator)                
                .pipe(impulse_calculator)
                .pipe(add_review_data))

In [ ]:
group_df.head()

In [ ]:
# for file in os.listdir(source_dir):
#     if file.startswith("NGS-"):
#         file_path = os.path.join(source_dir, file)
#         output_dir = "F:/Data/Processing_data/concussion_output/"

#         # Read the CSV into polars DF
#         df = pl.read_csv(file_path)
#         df = (df
#               .pipe(column_corrector)
#                 .pipe(angle_corrector)
#                 .pipe(body_builder_conc)
#                 .pipe(velocity_calculator)                
#                 .pipe(impulse_calculator)
#                 .pipe(add_review_data))
        
#         output_file_path = os.path.join(output_dir, file.replace(".csv", ".parquet"))

#         df.write_parquet(output_file_path)

#         print(f"Processed and saved: {output_file_path}")

# print("For fuck's sake that took a while. Finally done processing and saving the concussion files.")

In [ ]:
# def process_and_save_concussion_data():

#     for file in os.listdir(source_dir):
#         if file.startswith("NGS-"):
#             file_path = os.path.join(source_dir, file)
#             output_dir = "F:/Data/Processing_data/concussion_output/"

#             # Read the CSV into polars DF

#             df = pl.read_csv(file_path)
#             df = (df
#                 .pipe(column_corrector)
#                     .pipe(angle_corrector)
#                     .pipe(body_builder_conc)
#                     .pipe(velocity_calculator)                
#                     .pipe(impulse_calculator))
            
#             output_file_path = os.path.join(output_dir, file.replace(".csv", ".parquet"))

#             df.write_parquet(output_file_path)

#             print(f"Processed and saved: {output_file_path}")

#     print("For fuck's sake that took a while. Finally done processing and saving the concussion files.")

In [2]:
from QuantitativeCleaner import * 



In [3]:

process_and_save_concussion_data()

Memory usage of dataframe is 87.43 MB
Memory usage after optimization is: 35.77 MB
Decreased by 59.1%


c:\Users\justi\OneDrive\Desktop\Analytics\NFL_Data_Science\ET\DataHandler.py:140: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  df = df.with_columns(pl.col(col).cast(pl.Categorical))


Processed and saved: F:/Data/Processing_data/concussion_output/NGS-2016-post.parquet
Memory usage of dataframe is 95.19 MB
Memory usage after optimization is: 38.49 MB
Decreased by 59.6%
Processed and saved: F:/Data/Processing_data/concussion_output/NGS-2016-pre.parquet
Memory usage of dataframe is 790.15 MB
Memory usage after optimization is: 326.29 MB
Decreased by 58.7%
Processed and saved: F:/Data/Processing_data/concussion_output/NGS-2016-reg-wk1-6.parquet
Memory usage of dataframe is 690.97 MB
Memory usage after optimization is: 285.94 MB
Decreased by 58.6%
Processed and saved: F:/Data/Processing_data/concussion_output/NGS-2016-reg-wk13-17.parquet
Memory usage of dataframe is 760.78 MB
Memory usage after optimization is: 314.28 MB
Decreased by 58.7%
Processed and saved: F:/Data/Processing_data/concussion_output/NGS-2016-reg-wk7-12.parquet
Memory usage of dataframe is 94.12 MB
Memory usage after optimization is: 38.95 MB
Decreased by 58.6%
Processed and saved: F:/Data/Processing_da

# Combine Tracking Data

To do this, the columns will have to be the same in both the concussion and the injury tracking data. I will create a separate processing for this, so that the outputs 

In [5]:
import polars as pl
df = pl.read_parquet("F:/Data/Processing_data/concussion_output/NGS-2017-reg-wk13-17.parquet").head(1000)

In [6]:
df.head()

PlayKey,x,y,o,dir,GSISID,time,Angle_Diff,Position,Height_m,Weight_kg,Chest_rad_m,Displacement,Speed,vx,vy,omega_dir,omega_o,omega_diff,px,py,moment,moment_upper,p_magnitude,L_dir,L_diff,Jx,Jy,J_magnitude,torque,torque_internal
str,f32,f32,f32,f32,i32,i32,f32,cat,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""19714-585-120""",68.160004,10.81,-36.779999,-109.649994,19714,0,72.870003,"""P""",1.88,97.519997,0.191,null,null,null,null,null,null,null,null,null,0.296469,0.207528,null,null,null,null,null,null,null,null
"""19714-585-120""",68.07,10.87,-39.009995,-111.0,19714,0,71.989998,"""P""",1.88,97.519997,0.191,0.10817,1.081695,-0.90004,0.599995,-0.235621,-0.389208,0.153587,-87.771866,58.511478,0.296469,0.207528,105.486931,-0.069854,0.031874,null,null,null,null,null
"""19714-585-120""",68.040001,10.89,-40.610001,-109.609985,19714,0,69.0,"""P""",1.88,97.519997,0.191,0.036055,0.360547,-0.299988,0.200005,0.242603,-0.279254,0.521857,-29.254808,19.504446,0.296469,0.207528,35.160591,0.071924,0.1083,58.517059,-39.007034,70.326347,1.417784,0.764264
"""19714-585-120""",68.010002,10.91,-42.110001,-107.549988,19714,0,65.440002,"""P""",1.88,97.519997,0.191,0.036054,0.360542,-0.299988,0.199995,0.359538,-0.261799,0.621337,-29.254808,19.503515,0.296469,0.207528,35.160076,0.106592,0.128945,0.0,-0.000931,0.000931,0.346675,0.206449
"""19714-585-120""",68.129997,10.83,-36.800003,-110.670013,19714,0,73.870003,"""P""",1.88,97.519997,0.191,0.144218,1.442179,1.199951,-0.799999,-0.544547,0.926769,1.471316,117.019234,-78.015923,0.296469,0.207528,140.641327,-0.161441,0.30534,146.274048,-97.51944,175.801422,-2.68033,1.763946


In [7]:
review = clean_review()

Memory usage of dataframe is 0.00 MB
Memory usage after optimization is: 0.00 MB
Decreased by 36.9%


In [10]:

def add_review_data(df, review):

    df = df.join(
        review
        , on="PlayKey"
        , how="inner"    
        )
    
    return df

In [11]:
add_review_data(df, review)

PlayKey,x,y,o,dir,GSISID,time,Angle_Diff,Position,Height_m,Weight_kg,Chest_rad_m,Displacement,Speed,vx,vy,omega_dir,omega_o,omega_diff,px,py,moment,moment_upper,p_magnitude,L_dir,L_diff,Jx,Jy,J_magnitude,torque,torque_internal,Player_Activity_Derived,Primary_Impact_Type,Primary_Partner_GSISID,Primary_Partner_Activity_Derived
str,f32,f32,f32,f32,i32,i32,f32,cat,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,str,cat


In [12]:
len(review)

37

In [38]:
def create_concussion_review_df(review):
    """
    Create a new DataFrame from multiple Parquet files, including only rows where the PlayKey matches those in the review DataFrame.
    """
    import polars as pl # type: ignore
    import os

    ngs_dir = "F:/Data/Processing_data/concussion_output/"


    # Add the OpponentKey column
    review = review.with_columns(
        (pl.col("Primary_Partner_GSISID") + pl.col("PlayKey").str.slice(5)).alias("OpponentKey")
        )
    # Filter OpponentKey values that are longer than 12 characters
    
    # Extract PlayKey and OpponentKey values into lists
    playkey_list = review["PlayKey"].to_list()
    opponentkey_list = review["OpponentKey"].to_list()

    # Remove any "Unknown" GSISID opponents from the list, since it will be a nonsense PlayKey
    opponentkey_list = [key for key in opponentkey_list if key is not None and len(key) <= 12]

    # Combine both lists
    combined_keys = playkey_list + opponentkey_list


    # Initialize a list to store dataframes from each table
    dataframes = []

    # Iterate through the parquet files in the directory
    for file in os.listdir(ngs_dir):
        if file.startswith("NGS-"):
            file_path = os.path.join(ngs_dir, file)

            # Read into df
            df = pl.read_parquet(file_path)

            # Filter based on matching PlayKey values
            filtered_df = df.filter(pl.col('PlayKey').is_in(combined_keys))

            # Append to the dataframes
            dataframes.append(filtered_df)

    combined_df = pl.concat(dataframes)

    combined_df = combined_df.join(
        review
        , on='PlayKey'
        , how = 'left'
    )

    return combined_df




In [36]:
review = clean_review()

Memory usage of dataframe is 0.00 MB
Memory usage after optimization is: 0.00 MB
Decreased by 36.9%


In [39]:
cf = create_concussion_review_df(review)

In [40]:
cf.head()

PlayKey,x,y,o,dir,GSISID,time,Angle_Diff,Position,Height_m,Weight_kg,Chest_rad_m,Displacement,Speed,vx,vy,omega_dir,omega_o,omega_diff,px,py,moment,moment_upper,p_magnitude,L_dir,L_diff,Jx,Jy,J_magnitude,torque,torque_internal,Player_Activity_Derived,Primary_Impact_Type,Primary_Partner_GSISID,Primary_Partner_Activity_Derived,OpponentKey
str,f32,f32,f32,f32,i32,i32,f32,cat,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,str,cat,str
"""31023-29-538""",32.950001,38.700001,-117.579987,85.380005,31023,0,157.039993,"""WR""",1.88,90.699997,0.191,null,null,null,null,null,null,null,null,null,0.275736,0.193015,null,null,null,null,null,null,null,null,"""Tackling""","""Helmet-to-body""","""31941""","""Tackled""","""31941-29-538"""
"""31023-29-538""",32.98,38.720001,-119.970001,105.070007,31023,0,134.960007,"""WR""",1.88,90.699997,0.191,0.036055,0.360547,0.299988,0.200005,3.436552,-0.417137,3.853689,27.208893,18.140415,0.275736,0.193015,32.70166,0.94758,0.743819,null,null,null,null,null,"""Tackling""","""Helmet-to-body""","""31941""","""Tackled""","""31941-29-538"""
"""31023-29-538""",32.98,38.709999,-122.320007,90.539978,31023,0,147.139999,"""WR""",1.88,90.699997,0.191,0.010002,0.100021,0.0,-0.100021,-2.535969,-0.410154,2.125815,0.0,-9.071938,0.275736,0.193015,9.071938,-0.699257,0.410314,-27.208893,-27.212353,38.481632,-16.468365,-3.335054,"""Tackling""","""Helmet-to-body""","""31941""","""Tackled""","""31941-29-538"""
"""31023-29-538""",32.93,38.669998,-124.540009,48.880005,31023,0,173.419998,"""WR""",1.88,90.699997,0.191,0.064031,0.640312,-0.499992,-0.400009,-7.271037,-0.387464,6.883573,-45.349308,-36.28083,0.275736,0.193015,58.076317,-2.004883,1.328632,-45.349308,-27.208893,52.885574,-13.056264,9.183182,"""Tackling""","""Helmet-to-body""","""31941""","""Tackled""","""31941-29-538"""
"""31023-29-538""",32.900002,38.619999,-126.799988,40.029999,31023,0,166.830002,"""WR""",1.88,90.699997,0.191,0.058308,0.583082,-0.299988,-0.499992,-1.544617,-0.39444,1.150178,-27.208893,-45.349308,0.275736,0.193015,52.885574,-0.425906,0.222001,18.140415,-9.068478,20.280827,15.789774,-11.066308,"""Tackling""","""Helmet-to-body""","""31941""","""Tackled""","""31941-29-538"""


In [42]:
cf.filter(pl.col("PlayKey")=="31941-29-538")

PlayKey,x,y,o,dir,GSISID,time,Angle_Diff,Position,Height_m,Weight_kg,Chest_rad_m,Displacement,Speed,vx,vy,omega_dir,omega_o,omega_diff,px,py,moment,moment_upper,p_magnitude,L_dir,L_diff,Jx,Jy,J_magnitude,torque,torque_internal,Player_Activity_Derived,Primary_Impact_Type,Primary_Partner_GSISID,Primary_Partner_Activity_Derived,OpponentKey
str,f32,f32,f32,f32,i32,i32,f32,cat,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,str,cat,str
"""31941-29-538""",18.77,38.830002,-7.649994,60.259995,31941,0,67.910004,"""WR""",1.88,90.699997,0.191,null,null,null,null,null,null,null,null,null,0.275736,0.193015,null,null,null,null,null,null,null,null,null,null,null,null,null
"""31941-29-538""",18.83,38.700001,-8.690002,60.110001,31941,0,68.800003,"""WR""",1.88,90.699997,0.191,0.143179,1.43179,0.599995,-1.300011,-0.02618,-0.181516,0.155336,54.419514,-117.910965,0.275736,0.193015,129.863312,-0.007219,0.029982,null,null,null,null,null,null,null,null,null,null
"""31941-29-538""",18.9,38.59,-9.540009,59.820007,31941,0,69.360001,"""WR""",1.88,90.699997,0.191,0.130384,1.303844,0.699997,-1.100006,-0.050613,-0.148354,0.097741,63.489719,-99.770554,0.275736,0.193015,118.258644,-0.013956,0.018866,9.070206,18.140411,20.281595,-0.067371,-0.111166,null,null,null,null,null
"""31941-29-538""",19.0,38.48,-10.449997,59.550003,31941,0,70.0,"""WR""",1.88,90.699997,0.191,0.148661,1.486614,1.000004,-1.100006,-0.047125,-0.158823,0.111699,90.70034,-99.770554,0.275736,0.193015,134.835876,-0.012994,0.021559,27.210621,0.0,27.210621,0.009618,0.026939,null,null,null,null,null
"""31941-29-538""",19.09,38.380001,-11.399994,58.889999,31941,0,70.290001,"""WR""",1.88,90.699997,0.191,0.134535,1.345352,0.900002,-0.999985,-0.115193,-0.165806,0.050612,81.630135,-90.698616,0.275736,0.193015,122.023438,-0.031763,0.009769,-9.070206,9.071938,12.828432,-0.187689,-0.117905,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""31941-29-538""",35.98,33.360001,-64.160004,-112.910004,31941,32,48.75,"""WR""",1.88,90.699997,0.191,1.392049,13.920492,-5.699997,12.700005,-2.771585,2.961823,5.733408,-516.989685,1151.890381,0.275736,0.193015,1262.588501,-0.764225,1.106633,-11954.257812,-2648.439941,12244.121094,69.097771,-18.235016,null,null,null,null,null
"""31941-29-538""",21.18,42.93,105.580017,-169.769989,31941,32,84.650002,"""WR""",1.88,90.699997,0.191,17.624554,176.245544,-148.0,95.699997,-9.923939,29.625225,39.549164,-13423.599609,8679.989258,0.275736,0.193015,15985.469727,-2.736383,7.633578,-12906.610352,7528.098633,14941.648438,-19.721582,65.269455,null,null,null,null,null
"""31941-29-538""",20.01,42.91,120.039978,176.529999,31941,32,56.490002,"""WR""",1.88,90.699997,0.191,1.170171,11.70171,-11.700001,-0.200005,60.44075,2.523739,57.917011,-1061.190063,-18.140415,0.275736,0.193015,1061.345093,16.665665,11.178846,12362.40918,-8698.129883,15115.774414,194.020477,35.452682,null,null,null,null,null


# Concussion Summary Data

In [1]:
from QuantitativeCleaner import *

In [2]:
review = clean_review()

Memory usage of dataframe is 0.00 MB
Memory usage after optimization is: 0.00 MB
Decreased by 45.2%


In [3]:
create_concussion_review_df(review)